In [21]:
from google.cloud import language_v1
import os
import json

In [22]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '../Keys/exploratorium-docent-ai-2ea410926936.json'
client = language_v1.LanguageServiceClient()
type_ = language_v1.Document.Type.PLAIN_TEXT
language = "en"
encoding_type = language_v1.EncodingType.UTF8

In [23]:
def get_entities(grouping, fields):
    this_grouping_analysis = []

    for field in fields:
        path = os.path.join('data/split_data', grouping, field)

        this_field_analysis =[]

        for file in os.listdir(path):
            if file.startswith('.'):
                continue

            this_file_analysis = {}

            file_path = os.path.join(path, file)
            with open(file_path, 'r') as f:
                text_content = f.read()

            document = {'content': text_content, 'type_': type_, 'language': language}
            response = client.analyze_entities(request={'document': document, 'encoding_type': encoding_type})

            entities = []
            for entity in response.entities:
                this_entity = {}
                this_entity['name'] = entity.name
                this_entity['type'] = language_v1.Entity.Type(entity.type_).name
                this_entity['salience'] = entity.salience
                entities.append(this_entity)

            id = file.split('.')[0]
            this_file_analysis['id'] = id
            this_file_analysis['grouping'] = grouping
            this_file_analysis['category'] = field
            this_file_analysis['entities'] = entities

            this_field_analysis.append(this_file_analysis)

        this_grouping_analysis += this_field_analysis

    return this_grouping_analysis

In [24]:
exhibit_entities = get_entities('exhibits',
                                       ['byline', 'tagline', 'description', 'whats_going_on',
                                        'going_further', 'details'])

In [25]:
path = 'data/entities'

if not os.path.exists(path):
                os.makedirs(path)

exhibits_filepath = os.path.join(path, 'exhibits.json')

with open(exhibits_filepath, "w") as f:
    f.write(json.dumps(exhibit_entities, indent=2))

In [26]:
gallery_entities = get_entities('galleries', ['tagline', 'description', 'curator_statement'])

In [27]:
path = 'data/entities'

if not os.path.exists(path):
                os.makedirs(path)

galleries_filepath = os.path.join(path, 'galleries.json')

with open(galleries_filepath, "w") as f:
    f.write(json.dumps(gallery_entities, indent=2))